In [84]:
import subprocess

def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [85]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [86]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 7500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [87]:
# for i  in devices_log_list():
#     print(max(1,round(i)))


In [88]:
import concurrent.futures
import json
import math
from IPython.display import clear_output

def run_scenario(traffic_type, realistic_model, device_class, scenario_name):
    
    simTime = 4000
    appPeriod = 1200
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class)
            futures.append(future)



    for future in futures:
        data = json.loads(future.result())
        if float(data["ULPDR"]["Total"])>0:
            data_all.append(data)
            print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
        

    with open(scenario_name, 'w') as f:
        json.dump(data_all, f)
        clear_output(wait=True)

In [89]:
import threading

threads = []
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'A', "ULPDR_UNCONFIRMED_A")))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'A', "ULPDR_CONFIRMED_A")))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'A', "ULPDR_MIXED_A")))

In [90]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B")))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B")))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B")))


In [91]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C")))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C")))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C")))


In [92]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time), 'aditya.jagatha@trojans.dsu.edu')

48556.181051015854
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
